# MobileNetV3 fine-tuning

## Imports

In [1]:
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger

from src.data.loaders import KDEFDataModule
from src.data.split import generate_split
from src.models.trainer import EmotionClassifier

torch.set_float32_matmul_precision("high")

## Data Preparation

#### We discard images of angles `FullRight` and `FullLeft` and bugged files.

#### We perform a subject-wise split to ensure that images from the same subject do not appear in different sets.

In [2]:
generate_split()

Skipped buggy file: data\raw\AF01\AF01SUFR.JPG
Skipped buggy file: data\raw\AF10\AF10AFFR.JPG
Skipped buggy file: data\raw\AF11\AF11NEHL.JPG
Skipped buggy file: data\raw\AF20\AF20DIHL.JPG
Skipped buggy file: data\raw\AM25\AM25DIFL.JPG
Skipped buggy file: data\raw\AM34\AM34DIFR.JPG
Skipped buggy file: data\raw\BF13\BF13NEHR.JPG
Skipped buggy file: data\raw\BM21\BM21DIFL.JPG
Skipped buggy file: data\raw\BM22\BM22DIHL.JPG
Skipped buggy file: data\raw\BM24\BM24DIFL.JPG
Total images found: 2936
Unique subjects: 70

Split distribution:
split
train    2181
val       378
test      377
Name: count, dtype: int64

Saved master split to data\kdef_split.csv


### Model Training

In [3]:
data_module = KDEFDataModule(
    csv_file="data/kdef_split.csv",
    root_dir="data/raw",
    batch_size=32,
)

checkpoint_callback = ModelCheckpoint(
    dirpath="outputs/models",
    filename="mobilenet_v3_kdef-frozen-{epoch:02d}-{val_f1:.2f}",
    save_top_k=1,
    monitor="val_f1",
    mode="max",
)

model_frozen = EmotionClassifier(
    num_classes=7, learning_rate=1e-3, freeze_backbone=True
)

trainer = pl.Trainer(
    max_epochs=20,
    accelerator="auto",
    devices=1,
    logger=CSVLogger("outputs", name="logs"),
    callbacks=[checkpoint_callback],
    log_every_n_steps=10,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


INFO: Backbone frozen. Training classifier head only.


In [4]:
trainer.fit(model_frozen, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loaded data from data\raw:
Train: 2181, Val: 378, Test: 377


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name          ┃ Type                      ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model         │ MobileNetV3               │  4.2 M │ train │     0 │
│ 1 │ train_acc     │ MulticlassAccuracy        │      0 │ train │     0 │
│ 2 │ val_acc       │ MulticlassAccuracy        │      0 │ train │     0 │
│ 3 │ val_f1        │ MulticlassF1Score         │      0 │ train │     0 │
│ 4 │ val_precision │ MulticlassPrecision       │      0 │ train │     0 │
│ 5 │ val_recall    │ MulticlassRecall          │      0 │ train │     0 │
│ 6 │ test_acc      │ MulticlassAccuracy        │      0 │ train │     0 │
│ 7 │ test_f1       │ MulticlassF1Score         │      0 │ train │     0 │
│ 8 │ conf_mat      │ MulticlassConfusionMatrix │      0 │ train │     0 │
└───┴───────────────┴───────────────────────────┴────────┴───────┴───────┘

Trainable params: 1.2 M                                                                                            
Non-trainable params: 3.0 M                                                                                        
Total params: 4.2 M                                                                                                
Total estimated model params size (MB): 16                                                                         
Modules in train mode: 263                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

`Trainer.fit` stopped: `max_epochs=20` reached.


In [5]:
model_unfrozen = EmotionClassifier(
    num_classes=7, learning_rate=1e-3, freeze_backbone=False
)

checkpoint_callback = ModelCheckpoint(
    dirpath="outputs/models",
    filename="mobilenet_v3_kdef-unfrozen-{epoch:02d}-{val_f1:.2f}",
    save_top_k=1,
    monitor="val_f1",
    mode="max",
)

trainer = pl.Trainer(
    max_epochs=50,
    accelerator="auto",
    devices=1,
    logger=CSVLogger("outputs", name="logs"),
    callbacks=[checkpoint_callback],
    log_every_n_steps=10,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


In [6]:
trainer.fit(model_unfrozen, datamodule=data_module)

d:\studia\AffectiveAI\.venv\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:881: Checkpoint directory D:\studia\AffectiveAI\outputs\models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loaded data from data\raw:
Train: 2181, Val: 378, Test: 377


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name          ┃ Type                      ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model         │ MobileNetV3               │  4.2 M │ train │     0 │
│ 1 │ train_acc     │ MulticlassAccuracy        │      0 │ train │     0 │
│ 2 │ val_acc       │ MulticlassAccuracy        │      0 │ train │     0 │
│ 3 │ val_f1        │ MulticlassF1Score         │      0 │ train │     0 │
│ 4 │ val_precision │ MulticlassPrecision       │      0 │ train │     0 │
│ 5 │ val_recall    │ MulticlassRecall          │      0 │ train │     0 │
│ 6 │ test_acc      │ MulticlassAccuracy        │      0 │ train │     0 │
│ 7 │ test_f1       │ MulticlassF1Score         │      0 │ train │     0 │
│ 8 │ conf_mat      │ MulticlassConfusionMatrix │      0 │ train │     0 │
└───┴───────────────┴───────────────────────────┴────────┴───────┴───────┘

Trainable params: 4.2 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.2 M                                                                                                
Total estimated model params size (MB): 16                                                                         
Modules in train mode: 263                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

`Trainer.fit` stopped: `max_epochs=50` reached.
